# L5: Automate Event Planning

In this lesson, you will learn more about Tasks.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Crew, Task

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key,get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    )
)

In [6]:
 # Agent 2: Logistics Manager
logistics_manager = Agent(
    role='Logistics Manager',
    goal=(
        "Manage all logistics for the event "
        "including catering and equipmen"
    ),
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    )
)

In [7]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    )
)

## Creating Venue Pydantic Object

- Create a class `VenueDetails` using [Pydantic BaseModel](https://docs.pydantic.dev/latest/api/base_model/).
- Agents will populate this object with information about different venues by creating different instances of it.

In [8]:
from pydantic import BaseModel
# Define a Pydantic model for venue details 
# (demonstrating Output as Pydantic)
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

## Creating Tasks
- By using `output_json`, you can specify the structure of the output you want.
- By using `output_file`, you can get your output in a file.
- By setting `human_input=True`, the task will ask for human feedback (whether you like the results or not) before finalising it.

In [9]:
venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",  
      # Outputs the venue details as a JSON file
    agent=venue_coordinator
)

- By setting `async_execution=True`, it means the task can run in parallel with the tasks which come after it.

In [10]:
logistics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True, # to take human feedback before finalizing
    async_execution=True, # logistic depends on venue task but it doesnt related to marketing task so they both can run in paralle
    agent=logistics_manager
)

In [11]:
marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    async_execution=True,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent
)

## Creating the Crew

**Note**: Since you set `async_execution=True` for `logistics_task` and `marketing_task` tasks, now the order for them does not matter in the `tasks` list.

In [12]:
# Define the crew with agents and tasks
event_management_crew = Crew(
    agents=[venue_coordinator, 
            logistics_manager, 
            marketing_communications_agent],
    
    tasks=[venue_task, 
           logistics_task, 
           marketing_task],
    
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [15]:
event_details = {
    'event_topic': "Tech Innovation Conference",
    'event_description': "A gathering of tech innovators "
                         "and industry leaders "
                         "to explore future technologies.",
    'event_city': "Bangalore",
    'tentative_date': "2024-09-15",
    'expected_participants': 500,
    'budget': 20000,
    'venue_type': "Conference Hall"
}

**Note 1**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

**Note 2**: 
- Since you set `human_input=True` for some tasks, the execution will ask for your input before it finishes running.
- When it asks for feedback, use your mouse pointer to first click in the text box before typing anything.

In [16]:
result = event_management_crew.kickoff(inputs=event_details)

 [DEBUG]: == Working Agent: Venue Coordinator
 [INFO]: == Starting Task: Find a venue in Bangalore that meets criteria for Tech Innovation Conference.


> Entering new CrewAgentExecutor chain...
I need to start by researching potential venues in Bangalore that meet the criteria for the Tech Innovation Conference.

Action: Search the internet
Action Input: {"search_query": "venues in Bangalore for tech events"} 


Search results: Title: IT & Technology Events in Bengaluru - 10Times
Link: https://10times.com/bengaluru-in/technology
Snippet: IT & Technology Events in Bengaluru · World Machine Learning Summit · Bangalore Cloud & Datacenter Convention · GCC X Summit · World Test Engineering Summit.
---
Title: Venue - Bengaluru Tech Summit 2024
Link: https://www.bengalurutechsummit.com/BTS-venue.php
Snippet: Asia's Largest Technology Event, Bengaluru Tech Summit (BTS) organized by ... Venue. Event Location. Bangalore Palace Bengaluru, Karnataka, India.
---
Title: The 372 Best Event Venues an

 


Search results: Title: House Party Caterers in bangalore - CaterNinja
Link: https://www.caterninja.com/bangalore/caterers/category/house-party-caterers
Snippet: Looking for fresh, friendly and professional Catering for house parties in bangalore | We have curated a bunch of best Home Party Caterers for small groups ...
---
Title: Bhandary's Kitchen: #1 Catering Services Provider in Bangalore
Link: https://bhandaryskitchen.com/
Snippet: Bhandary's Kitchen offers a wide variety of catering services in Bangalore with amazing food to fit both your event and your budget.
---
Title: Top Caterers near Bangalore - Best Food Catering Services - Justdial
Link: https://www.justdial.com/Bangalore/Caterers/nct-10083293
Snippet: Popular Catering Companies · Taj Sats Air Catering Limited · Blue Sea Catering and Banquets · Vinayaka Catering · Caterninja · Bismillah Caterers · Friends ...
---
Title: Best 20 Caterers near me in Bangalore | For Weddings & Parties
Link: https://www.wedmegood.com/vendo

Thought: I now can give a great answer
Final Answer: After conducting a search, I found that Bangalore Palace in Vasanth Nagar, Bangalore has a banquet hall with a guest capacity ranging from 850 to 2250. Unfortunately, I was unable to find specific booking status information for the Tech Innovation Conference.

> Finished chain.
 

After conducting a search, I found that Bangalore Palace in Vasanth Nagar, Bangalore has a banquet hall with a guest capacity ranging from 850 to 2250. Unfortunately, I was unable to find specific booking status information for the Tech Innovation Conference.

Thought: 
I should continue gathering information and devising a marketing strategy to attract potential attendees to the Tech Innovation Conference.

Action: Read website content
Action Input: {"website_url": "https://www.techinnovationconference.com"} 

I encountered an error while trying to use the tool. This was the error: HTTPSConnectionPool(host='www.techinnovationconference.com', port=443): Max

Final Answer: 
Based on the search results, some catering options near Bangalore Palace for the event on 2024-09-15 include:
1. AS Wedding & Events
2. Zaika Caterers
3. Vasavi Catering Service
4. Basheer's Classic Caterers
5. Sri Mayyia Caterers

These caterers offer a variety of services suitable for different types of events. It would be beneficial to reach out to them to discuss catering options and finalize the arrangements for the event at Bangalore Palace.

> Finished chain.
 

Based on the search results, some catering options near Bangalore Palace for the event on 2024-09-15 include:
1. AS Wedding & Events
2. Zaika Caterers
3. Vasavi Catering Service
4. Basheer's Classic Caterers
5. Sri Mayyia Caterers

These caterers offer a variety of services suitable for different types of events. It would be beneficial to reach out to them to discuss catering options and finalize the arrangements for the event at Bangalore Palace.

Thought: 
I need to ensure that all logistics arrangements

Exception in thread Thread-6 (_execute):
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/usr/local/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.11/site-packages/crewai/task.py", line 182, in _execute
    result = agent.execute_task(
             ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/crewai/agent.py", line 221, in execute_task
    result = self.agent_executor.invoke(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/langchain/chains/base.py", line 163, in invoke
    raise e
  File "/usr/local/lib/python3.11/site-packages/langchain/chains/base.py", line 153, in invoke
    self._call(inputs, run_manager=run_manager)
  File "/us

Thought: 
I now have gathered valuable insights on promotion strategies and details of similar innovation conferences that can help in marketing the Tech Innovation Conference effectively.

Final Answer: 

## Marketing Activities and Attendee Engagement Report

### Marketing Strategies:
1. **Target Audience Identification:** 
   - Utilize data analytics to identify the target audience segments most likely to be interested in the Tech Innovation Conference.
   - Tailor marketing messages and content based on the preferences and needs of these segments.

2. **Differentiated Audience Approach:**
   - Develop customized marketing approaches for different audience groups to maximize engagement and attendance.
   - Create personalized communication strategies to connect with diverse attendee personas.

3. **Compelling Branding and Messaging:**
   - Design a clear and compelling branding strategy for the conference to generate interest and establish a strong event identity.
   - Communicate t

- Display the generated `venue_details.json` file.

In [17]:
import json
from pprint import pprint

with open('venue_details.json') as f:
   data = json.load(f)

pprint(data)

{'address': 'No.11/6, NITON, Block "C", Second Floor, Palace Road, Bengaluru - '
            '560001, Karnataka, India',
 'booking_status': 'Not specified',
 'capacity': 0,
 'name': 'Bangalore Palace'}


- Display the generated `marketing_report.md` file.

**Note**: After `kickoff` execution has successfully ran, wait an extra 45 seconds for the `marketing_report.md` file to be generated. If you try to run the code below before the file has been generated, your output would look like:

```
marketing_report.md
```

If you see this output, wait some more and than try again.

In [18]:
from IPython.display import Markdown
Markdown("marketing_report.md")

## Marketing Activities and Attendee Engagement Report

### Marketing Strategies:
1. **Target Audience Identification:** 
   - Utilize data analytics to identify the target audience segments most likely to be interested in the Tech Innovation Conference.
   - Tailor marketing messages and content based on the preferences and needs of these segments.

2. **Differentiated Audience Approach:**
   - Develop customized marketing approaches for different audience groups to maximize engagement and attendance.
   - Create personalized communication strategies to connect with diverse attendee personas.

3. **Compelling Branding and Messaging:**
   - Design a clear and compelling branding strategy for the conference to generate interest and establish a strong event identity.
   - Communicate the value proposition of the conference through impactful messaging across various marketing channels.

4. **Interactive Conference Website:**
   - Build a user-friendly and engaging conference website that provides detailed information about the event, agenda, speakers, and registration.
   - Implement interactive features such as live chat support, virtual tours, and multimedia content to enhance attendee experience.

5. **Social Media Promotion:**
   - Leverage social media platforms to promote the conference, engage with potential attendees, and create buzz around the event.
   - Utilize targeted advertising, influencer partnerships, and user-generated content to reach a wider audience and drive registration.

### Attendee Engagement:
1. **Early Bird Registration Discounts:**
   - Offer early bird registration discounts and exclusive perks to incentivize attendees to register early and secure their spot at the conference.
   - Create urgency through limited-time offers and countdown timers to drive early registrations.

2. **Networking Opportunities:**
   - Facilitate networking sessions, virtual meetups, and interactive workshops to encourage attendee interaction and collaboration.
   - Provide matchmaking tools and personalized networking recommendations to help attendees connect with like-minded professionals.

3. **Engaging Session Formats:**
   - Curate a diverse range of session formats including keynote speeches, panel discussions, hands-on workshops, and interactive Q&A sessions.
   - Incorporate innovative presentation styles, audience polls, and interactive elements to keep attendees engaged and actively participating.

4. **Feedback Mechanisms:**
   - Implement feedback mechanisms such as surveys, polls, and live feedback channels to gather real-time insights from attendees.
   - Use attendee feedback to continuously improve the conference experience, address concerns, and enhance overall satisfaction.

5. **Post-Event Follow-Up:**
   - Send post-event surveys, thank you notes, and exclusive content to attendees to maintain engagement and gather feedback.
   - Nurture relationships with attendees through post-event communication, networking opportunities, and exclusive offers for future events.

By implementing these marketing strategies and attendee engagement tactics, the Tech Innovation Conference aims to attract a diverse audience of over 500 potential attendees and create a memorable and impactful event experience.